# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [31]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import os
# Import API key
from api_keys import geoapify_key

In [32]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")


# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,saint-louis,16.3333,-15.0000,98.17,26,5,3.51,SN,1697659249
1,1,ushuaia,-54.8000,-68.3000,60.46,42,0,16.11,AR,1697659223
2,2,faya-largeau,17.9257,19.1043,83.48,18,72,13.02,TD,1697659249
3,3,harnosand,62.6323,17.9379,32.00,88,6,6.67,SE,1697659250
4,4,port-aux-francais,-49.3500,70.2167,38.59,97,100,19.26,TF,1697659250


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [33]:

# Configure the map plot
fig1 = city_data_df.hvplot.points('Lng', 'Lat', geo=True, tiles='OSM', size='Humidity')

# Display the map
fig1



:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [21]:
# Narrow down cities that fit criteria and drop any results with null values
filtered_weather_df=city_data_df[(city_data_df['Max Temp'] < 80) &
                             (city_data_df['Max Temp'] > 70) &
                             (city_data_df['Wind Speed'] < 10) &
                             (city_data_df['Cloudiness'] == 0)]

filtered_weather_df=filtered_weather_df.dropna()
 # drop any rows will null values

filtered_weather_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
61,61,raebareli,26.2167,81.2333,71.29,60,0,5.50,IN,1697659272
146,146,khagaria,25.5000,86.4833,72.10,71,0,5.91,IN,1697659285
196,196,holualoa,19.6228,-155.9522,78.78,76,0,5.01,US,1697659294
218,218,siwa oasis,29.2041,25.5195,77.04,29,0,4.70,EG,1697659298
276,276,port hedland,-20.3167,118.5667,73.45,31,0,9.22,AU,1697659305
300,300,karratha,-20.7377,116.8463,79.29,45,0,9.80,AU,1697659299
310,310,nampula,-15.1165,39.2666,78.75,57,0,8.05,MZ,1697659309
372,372,chichawatni,30.5333,72.7000,70.95,49,0,1.68,PK,1697659317
388,388,najran,17.4924,44.1277,74.05,26,0,3.74,SA,1697659319
391,391,luena,-11.7833,19.9167,78.98,53,0,4.61,AO,1697659319


### Step 3: Create a new DataFrame called `hotel_df`.

In [22]:

 # Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = filtered_weather_df.copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df   

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
61,61,raebareli,26.2167,81.2333,71.29,60,0,5.50,IN,1697659272,
146,146,khagaria,25.5000,86.4833,72.10,71,0,5.91,IN,1697659285,
196,196,holualoa,19.6228,-155.9522,78.78,76,0,5.01,US,1697659294,
218,218,siwa oasis,29.2041,25.5195,77.04,29,0,4.70,EG,1697659298,
276,276,port hedland,-20.3167,118.5667,73.45,31,0,9.22,AU,1697659305,
300,300,karratha,-20.7377,116.8463,79.29,45,0,9.80,AU,1697659299,
310,310,nampula,-15.1165,39.2666,78.75,57,0,8.05,MZ,1697659309,
372,372,chichawatni,30.5333,72.7000,70.95,49,0,1.68,PK,1697659317,
388,388,najran,17.4924,44.1277,74.05,26,0,3.74,SA,1697659319,
391,391,luena,-11.7833,19.9167,78.98,53,0,4.61,AO,1697659319,


In [34]:
geoapify_key

'9e58be6db7a04bd8bb07b687a9791e3d'

### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [35]:
# Set parameters to search for a hotel
radius = 10000
params = {"categories": "accommodation.hotel",
          "limit": 2,
          "apiKey": geoapify_key}



# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    # YOUR CODE HERE
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f'circle:{row["Lng"]},{row["Lat"]},{radius}'
    params["bias"] = f'proximity:{row["Lng"]},{row["Lat"]}'
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    print(name_address)
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except Exception as E:
        print(E)
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
{'type': 'FeatureCollection', 'features': []}
list index out of range
raebareli - nearest hotel: No hotel found
{'type': 'FeatureCollection', 'features': []}
list index out of range
khagaria - nearest hotel: No hotel found
{'type': 'FeatureCollection', 'features': [{'type': 'Feature', 'properties': {'name': 'Kona Hotel', 'country': 'United States', 'country_code': 'us', 'state': 'Hawaii', 'county': 'Hawaiʻi County', 'postcode': '96725', 'suburb': 'Holualoa CDP', 'street': 'Mamalahoa Highway', 'lon': -155.9492669, 'lat': 19.6202624, 'state_code': 'HI', 'formatted': 'Kona Hotel, Mamalahoa Highway, Holualoa CDP, HI 96725, United States of America', 'address_line1': 'Kona Hotel', 'address_line2': 'Mamalahoa Highway, Holualoa CDP, HI 96725, United States of America', 'categories': ['accommodation', 'accommodation.hotel'], 'details': ['details'], 'datasource': {'sourcename': 'openstreetmap', 'attribution': '© OpenStreetMap contributors', 'license': 'Open Database Licenc

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
61,61,raebareli,26.2167,81.2333,71.29,60,0,5.50,IN,1697659272,No hotel found
146,146,khagaria,25.5000,86.4833,72.10,71,0,5.91,IN,1697659285,No hotel found
196,196,holualoa,19.6228,-155.9522,78.78,76,0,5.01,US,1697659294,Kona Hotel
218,218,siwa oasis,29.2041,25.5195,77.04,29,0,4.70,EG,1697659298,فندق الكيلانى
276,276,port hedland,-20.3167,118.5667,73.45,31,0,9.22,AU,1697659305,The Esplanade Hotel
300,300,karratha,-20.7377,116.8463,79.29,45,0,9.80,AU,1697659299,Karratha International Hotel
310,310,nampula,-15.1165,39.2666,78.75,57,0,8.05,MZ,1697659309,Hotel Lurio
372,372,chichawatni,30.5333,72.7000,70.95,49,0,1.68,PK,1697659317,No hotel found
388,388,najran,17.4924,44.1277,74.05,26,0,3.74,SA,1697659319,No hotel found
391,391,luena,-11.7833,19.9167,78.98,53,0,4.61,AO,1697659319,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [41]:
#configure the map plot
map_plot = hotel_df.hvplot.points(
    "Lng", 
    "Lat", 
    geo = True, 
    size = "Humidity",
    scale = 1,
    color = "City",
    alpha = 0.5,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    hover_cols = ["Hotel Name", "Country"]
)

# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)